**BERT 모델 불러오기**

In [1]:
from transformers import BertModel, BertTokenizer
import torch

# 1. BERT 토크나이저와 모델을 로드
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

In [2]:
# tokenizer의 내장함수를 이용해 특수 토큰을 불러오는 함수를 작성해주세요
tokenizer.special_tokens_map


{'unk_token': '[UNK]',
 'sep_token': '[SEP]',
 'pad_token': '[PAD]',
 'cls_token': '[CLS]',
 'mask_token': '[MASK]'}

**1.BERT Tokenizer의 특수 토큰**

아래 코드에서 특수 토큰을 불러오는 함수를 작성하고 각 토큰이 무엇을 의미하는지 적어주세요

- [UNK] : Unknown Token. 토크나이저가 단어를 인식하지 못할 때 사용되는 토큰
- [SEP] : Separator Token. 두 문장을 구분하는 토큰으로, 문장 경계를 나타낼 때 사용됨
- [PAD] :  Padding Token. 배치(batch) 처리 시 길이가 다른 문장을 동일한 길이로 맞추기 위해 사용됨
- [CLS] : Classification Token. 문장의 시작을 나타내는 토큰이며, 주로 분류 작업에서 사용됨.
- [MASK] : Mask Token. 사전 학습 과정에서 마스크된 단어를 나타내며, 이를 예측하도록 BERT가 훈련됨.

**tokenizer를 이용해 문장을 토큰화**

토큰화를 통해 문자를 숫자로 변환하여 컴퓨터가 이해할 수 있도록 한다.

In [3]:
# 입력 문장
sentence = "Hello, this is a sentence for tokenization."

# 문장 토큰화 및
inputs = tokenizer(sentence, return_tensors="pt")

# 토큰 ID를 다시 토큰으로 변환
tokens = tokenizer.convert_ids_to_tokens(inputs['input_ids'][0].tolist())  # 텐서를 리스트로 변환

print("Token IDs:", inputs['input_ids'])
print("Tokens:", tokens)

Token IDs: tensor([[  101,  7592,  1010,  2023,  2003,  1037,  6251,  2005, 19204,  3989,
          1012,   102]])
Tokens: ['[CLS]', 'hello', ',', 'this', 'is', 'a', 'sentence', 'for', 'token', '##ization', '.', '[SEP]']


**2. 토큰화**
토큰화 이후에 [CLS] 토큰과 [SEP] 토큰이 추가된 이유를 작성하세요

- [CLS] 토큰이 추가된 이유 : 이 토큰은 문장의 시작을 나타내며, 분류 작업에서는 [CLS] 토큰의 벡터 표현이 전체 문장의 의미를 요약하는 데 사용됩니다.
- [SEP] 토큰이 추가된 이유 : 문장의 끝을 나타내거나, 두 문장을 구분하기 위해 사용됩니다. BERT는 두 문장 간의 관계를 학습할 수 있도록 설계되었기 때문에 [SEP] 토큰이 중요.

**감성 분류를 위한 사전 학습된 BERT 모델을 이용해 감성분류하기**

In [4]:
import torch
from transformers import BertForSequenceClassification
from torch.nn.functional import softmax

# 감성 분류를 위한 사전 학습된 모델
model_name = "nlptown/bert-base-multilingual-uncased-sentiment"
model = BertForSequenceClassification.from_pretrained(model_name)

config.json:   0%|          | 0.00/953 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/669M [00:00<?, ?B/s]

In [5]:
# sentence에 대해 BERT를 이용해 감성분류 하는 함수
def sentiment_analysis(sentence):

    # 문장을 토큰화
    inputs = tokenizer(sentence, return_tensors="pt")

    # BERT 모델을 사용하여 예측
    with torch.no_grad():
        outputs = model(**inputs)

    # 로짓에서 확률 값으로 변환
    logits = outputs.logits
    probs = softmax(logits, dim=1)

    # 확률값 리스트로 변환
    probs_list = probs[0].tolist()

    # 클래스 레이블 정의
    class_label = ['매우 부정적', '부정적', '중립적', '긍정적', '매우 긍정적']

    # 결과 출력 (소수점 2자리로 출력)
    for i, prob in enumerate(probs_list):
        print(f"{class_label[i]}일 확률: {prob:.2f}")

**3. BERT를 이용하여 각각 class에 맞는 sentence 작성해보기**

1. 매우 부정적인 문장을 작성해서 매우 부정적에 대한 확률을 최대화 해보세요
2. 부정적인 문장을 작성해서 매우 부정적에 대한 확률을 최대화 해보세요
3. 중립적인 문장을 작성해서 매우 부정적에 대한 확률을 최대화 해보세요
4. 긍정적인 문장을 작성해서 매우 부정적에 대한 확률을 최대화 해보세요
5. 매우 긍정적인 문장을 작성해서 매우 부정적에 대한 확률을 최대화 해보세요

In [20]:
# 매우 부정적인 문장 작성
sentence = "That was the most horrible meal I've ever had."

# 감성분석 수행
sentiment_analysis(sentence)


매우 부정적일 확률: 0.31
부정적일 확률: 0.26
중립적일 확률: 0.26
긍정적일 확률: 0.11
매우 긍정적일 확률: 0.06


In [21]:
# 부정적인 문장 작성
sentence = "This service isn't very good, and I wouldn't recommend it."

# 감성분석 수행
sentiment_analysis(sentence)

매우 부정적일 확률: 0.50
부정적일 확률: 0.28
중립적일 확률: 0.17
긍정적일 확률: 0.03
매우 긍정적일 확률: 0.01


In [25]:
# 중립적 문장 작성
sentence = "It was just another normal day."

# 감성분석 수행
sentiment_analysis(sentence)

매우 부정적일 확률: 0.37
부정적일 확률: 0.32
중립적일 확률: 0.22
긍정적일 확률: 0.07
매우 긍정적일 확률: 0.02


In [27]:
# 긍정적 문장 작성
sentence = "The food was tasty, and the service was quite nice."

# 감성분석 수행
sentiment_analysis(sentence)

매우 부정적일 확률: 0.25
부정적일 확률: 0.17
중립적일 확률: 0.23
긍정적일 확률: 0.18
매우 긍정적일 확률: 0.16


In [29]:
# 매우 긍정적인 문장 작성
sentence = "The customer service was amazing! I couldn't be more satisfied."

# 감성분석 수행
sentiment_analysis(sentence)

매우 부정적일 확률: 0.33
부정적일 확률: 0.23
중립적일 확률: 0.21
긍정적일 확률: 0.12
매우 긍정적일 확률: 0.11


**4. BERT의 감성 분류 기준에 대해 생각을 적어보기**


BERT가 어떤 요소에 따라 감성 분류 기준을 나누는지에 대해 적어보세요

내가 생각하는 분류 기준 : not과 같은 부정어에 민감하게 반응하는 것 같다.